In [ ]:
# ------------------------------------------------------------------------------
# Author: SeongIl Kim
# Date: 2025-04-12
# Description: Automated crawling of Busan Port cargo throughput data using Selenium
#              and export to CSV file format
# ------------------------------------------------------------------------------

from selenium import webdriver                           # For selenium webdriver (셀레니움 웹 드라이버 실행)
from selenium.webdriver.chrome.service import Service    # For setting Chrome driver service (크롬 드라이버 서비스 설정)
from selenium.webdriver.common.by import By              # For searching web elements (웹 요소 탐색 전략 클래스)
from selenium.webdriver.common.keys import Keys          # For inputting key to driver (드라이버에 키 입력)
import pandas as pd                                      # For handling and saving data (데이터 처리 및 저장)
import time                                              # For latency of driver (드라이버에 지연 처리)

# ------------------------------------------------------------------------------
# Chrome 브라우저 옵션 및 드라이버 설정
# ------------------------------------------------------------------------------
myOption = webdriver.ChromeOptions()
myOption.add_argument("no-sandbox")     # 리눅스 환경 등에서 샌드박스 비활성화

chrome_path = "../chromedriver-win64/chromedriver-win64/chromedriver.exe"
myChrome = webdriver.Chrome(service=Service(chrome_path), options=myOption)
myChrome.maximize_window()

# ------------------------------------------------------------------------------
# 대상 웹사이트 접속 및 로그인 처리
# ------------------------------------------------------------------------------
myUrl = 'https://www.chainportal.co.kr/portstat/nexacro/index.html?screenid=screen_stat'
myChrome.get(myUrl)
time.sleep(2)
print("URL 이동 완료")

inputs = myChrome.find_elements(By.CLASS_NAME, 'nexainput')
time.sleep(2)

inputs[2].send_keys("zoqxls98")           # ID 입력
time.sleep(1)
inputs[2].send_keys(Keys.TAB)             # PW 입력 필드로 이동
time.sleep(1)
inputs[3].send_keys('Rlatjddlf527&')      # PW 입력
time.sleep(1)
inputs[3].send_keys(Keys.ENTER)           # 로그인
time.sleep(1)
print("로그인 완료")

# ------------------------------------------------------------------------------
# Port-MIS > 월별 화물처리실적 페이지 진입
# ------------------------------------------------------------------------------
selectboxes = myChrome.find_elements(By.CSS_SELECTOR, 'div.GridCellControl.cell.cell_LF_1Depth.dummy.dummy.dummy')
time.sleep(1)
selectboxes[5].click()
selectboxes[5].find_element(
    By.XPATH, '//*[@id="mainframe.frameIndex.form.divFrameLeft.form.divLeft.form.grdTree.body.gridrow_4.cell_4_1:text"]'
).click()
time.sleep(1)
print("월별 화물처리실적 클릭 완료")

# ------------------------------------------------------------------------------
# 연도별 화물처리실적 수집 함수 정의
# ------------------------------------------------------------------------------
def getCT(year):
    # 년도 입력 필드 클릭 및 값 입력
    myChrome.find_element(By.CSS_SELECTOR, 'div.Edit.req>input.nexainput').click()
    time.sleep(1)
    myChrome.find_element(By.CSS_SELECTOR, 'div.Edit.req>input.nexainput').send_keys(Keys.BACKSPACE * 4, year)
    time.sleep(1)

    # 조회 버튼 클릭
    myChrome.find_element(By.CSS_SELECTOR, 'div.Button.btn_WF_Search').click()
    time.sleep(1)

    # 연도 및 월 데이터 생성
    Year = [year] * 12
    Month = [i for i in range(1, 13)]
    CT = []

    # 테이블 데이터 추출
    table = myChrome.find_elements(By.CSS_SELECTOR, 'div.GridBandControl.body>div.nexacontainer>div.nexainnercontainer')[3]
    values = table.find_elements(By.CSS_SELECTOR, 'div.nexacontentsbox.nexarightalign')

    # '합계' 값만 추출
    for i in range(0, len(values), 5):
        num_text = values[i].text
        num = int(num_text.replace(",", "")[:-3])  # 콤마 제거 + 소수점 제외
        CT.append(num)

    # 데이터프레임 생성 및 반환
    df = pd.DataFrame({"Year": Year, "Month": Month, "CT(Cargo Throughput)": CT})
    return df

# ------------------------------------------------------------------------------
# 연도별 데이터 수집 및 통합
#  크롤링 과정
#   1. 연도 리스트 생성(2013 - 2024)
#   2. 빈 데이터 프레임 생성
#   3. 반복문을 이용해 연도 리스트의 각 연도별 데이터 수집
#   4. 빈 데이터 프레임에 하나씩 병합
# ------------------------------------------------------------------------------
years = [i for i in range(2013, 2025)]
busanCT = pd.DataFrame()

for year in years:
    df = getCT(year)
    busanCT = pd.concat([busanCT, df], ignore_index=True)
    print(f"{year}년 화물처리실적값 추출 완료")

# ------------------------------------------------------------------------------
# 크롬 브라우저 종료 및 CSV 파일 저장
# ------------------------------------------------------------------------------
myChrome.quit()
print('데이터 수집 완료')
busanCT.to_csv("./useData/busanCT(화물처리실적).csv", index=False, encoding='utf-8-sig')
print('데이터 저장 완료')

URL 이동 완료
로그인 완료
월별 화물처리실적 클릭 완료
2013년 화물처리실적값 추출 완료
2014년 화물처리실적값 추출 완료
2015년 화물처리실적값 추출 완료
2016년 화물처리실적값 추출 완료
2017년 화물처리실적값 추출 완료
2018년 화물처리실적값 추출 완료
2019년 화물처리실적값 추출 완료
2020년 화물처리실적값 추출 완료
2021년 화물처리실적값 추출 완료
2022년 화물처리실적값 추출 완료
2023년 화물처리실적값 추출 완료
2024년 화물처리실적값 추출 완료
데이터 수집 완료
데이터 저장 완료
